# CNN for Image Classification

### Import Libraries

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import random_split, ConcatDataset

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torcheval.metrics.functional import (multiclass_accuracy, 
                                          multiclass_confusion_matrix, 
                                          multiclass_precision, 
                                          multiclass_recall)
from sklearn.metrics import ConfusionMatrixDisplay, recall_score, precision_score, accuracy_score

### Load the data

In [ ]:
# Batch size for training, validation and testing datasets
batch_size = 64

# Percentages for training, validation and training sets
train_split = 0.6
valid_split = 0.2
test_split = 0.2

In [ ]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                ])

In [ ]:
train_data = datasets.CIFAR10('./data', train=True, transform=transform, download=True)
test_data = datasets.CIFAR10('./data', train=False, transform=transform, download=True)

In [ ]:
# Concatenate the datasets
full_dataset = ConcatDataset([train_data, test_data])

len_full_dataset = len(full_dataset)
print("Full dataset length", len_full_dataset)

In [ ]:
# Split data into training, validation and test datasets

# Seed the generator to achieve the same splits everytime
split_generator = torch.Generator().manual_seed(42) 

train_size = int(np.floor(train_split * len_full_dataset))
valid_size = int(np.floor(valid_split * len_full_dataset))
test_size = int(np.floor(test_split * len_full_dataset))

train_dataset, valid_dataset, test_dataset = random_split(full_dataset, 
                                                               [train_size, valid_size, test_size], 
                                                               split_generator)

In [ ]:
print("Train dataset length: ", len(train_dataset))
print("Validation dataset length: ", len(valid_dataset))
print("Test dataset length: ", len(test_dataset))

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size, shuffle=True)

In [ ]:
# Image classes
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

### Visualize a batch from dataset

In [ ]:
def imshow(img):
    # Un-normalize the images
    img = img * 0.5 + 0.5
    plt.imshow(np.transpose(img, (1, 2, 0))) # Change the shape

In [ ]:
images, labels = next(iter(train_loader))
images = images.numpy()

fig = plt.figure(figsize=(25, 4))
for idx in np.arange(20):
    ax = fig.add_subplot(2, 20//2, idx+1, xticks=[], yticks=[])
    imshow(images[idx])
    ax.set_title(classes[labels[idx]])

In [ ]:
print("Batch Shape: ", images.shape)

### Define the Network Architecture

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        # First convolution block
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 32, 3, padding=1)
        
        # Second covolution block
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv4 = nn.Conv2d(64, 64, 3, padding=1)
        
        # Third convolution block
        self.conv5 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv6 = nn.Conv2d(128, 128, 3, padding=1)
        
        # Max pooling layer
        self.pool = nn.MaxPool2d(2, 2)
        
        # Linear layer (4 * 4 * 128 -> 128)
        self.fc1 = nn.Linear(4 * 4 * 128, 128)
        self.fc2 = nn.Linear(128, 10)
        
        # Dropout layer
        self.dropout1 = nn.Dropout(0.2)
        self.dropout2 = nn.Dropout(0.3)
        self.dropout3 = nn.Dropout(0.4)
        self.dropout4 = nn.Dropout(0.5)
        
    def forward(self, x):
        
        # First block
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = self.dropout1(x)
        
        # Second block
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.pool(x)
        x = self.dropout2(x)
        
        # Third block
        x = F.relu(self.conv5(x))
        x = F.relu(self.conv6(x))
        x = self.pool(x)
        x = self.dropout3(x)
        
        # Flattern
        x = x.view(-1, 4 * 4 * 128)
        
        # Linear layer
        x = F.relu(self.fc1(x))
        x = self.dropout4(x)

        x = self.fc2(x)
        
        return x

In [ ]:
# create the CNN
model = Net()
model

In [ ]:
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('Training on CPU ...')
else:
    print('Training on GPU ...')

In [ ]:
if train_on_gpu:
    model.cuda()

### Loss Function & Optimizer

In [ ]:
# Categorical cross entrophy loss
criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=0.001)